Load sparkmagic before start


In [1]:
%load_ext sparkmagic.magics

In [9]:
from azureml.core import Workspace, Dataset

In [11]:
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B66CZHHV6 to authenticate.


WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/fd799da1-bfc1-4234-a91c-72b3a1cb9e26', 'tenant_id': 'fd799da1-bfc1-4234-a91c-72b3a1cb9e26'}' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: 65ae0088-e204-4d96-b626-35ea280e6600\r\nCorrelation ID: 8aafd07a-113a-48a9-a71a-a7134c7c8360\r\nTimestamp: 2020-03-18 03:33:39Z","error_codes":[50076],"timestamp":"2020-03-18 03:33:39Z","trace_id":"65ae0088-e204-4d96-b626-35ea280e6600","correlation_id":"8aafd07a-113a-48a9-a71a-a7134c7c8360","error_uri":"https://login.microsoftonline.com/error?code=50076","suberror":"basic_action"}'


Interactive authentication successfully completed.


In [ ]:
dsetdata = 'noaa-isd-files'
data_url = 'https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather'

if dsetdata not in ws.datasets:
    os.system('sudo chmod 777 /mnt')
    for year in range(2008, 2020+1):
        ds = Dataset.File.from_files(f'{data_url}/year={year}/month=*/*.parquet', validate=False)
        print('Downloading...')
        %time ds.download(f'/mnt/data/isd/year={year}', overwrite=True)
    print('Uploading...')
    %time ws.get_default_datastore().upload('/mnt/data/isd', '/noaa-isd', show_progress=False)
    ds = Dataset.File.from_files((ws.get_default_datastore(), '/noaa-isd/**/*.parquet'))
    ds = ds.register(ws, dsetdata)

Downloading...


### Start Spark Session

In [2]:
# You can change to any Synapse workspace you can access
# Please expect long waiting time, especially at first run

%spark start --workspace sparky --sparkpool sparky

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B67Q4S768 to authenticate.
Starting Spark application ...
SparkSession available as 'spark'.


### Working with Dataset

In [7]:
%%spark

# get workspace
from azureml.core import Workspace
subscription_id = '6560575d-fa06-4e7d-95fb-f962e74efd7a'
resource_group = 'cody-westus2-rg'
workspace_name = 'AzureML'

ws = Workspace(subscription_id, resource_group, workspace_name)
ws

Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-westus2-rg')

In [6]:
%%spark

sc

<SparkContext master=yarn appName=aml21835>

### Show Session Details

In [ ]:
%spark info

### Stop Session

In [ ]:
%spark stop

### Override Session Config

In [ ]:
%%spark config
{
    "driverMemory":"8g",
    "driverCores":2,
    "executorMemory":"8g",
    "executorCores":2,
    "numExecutors":2
}

### Example: RDD operation

In [ ]:
%%spark

import time
b=sc.broadcast([3,5]) #Creating a broadcast variable available on all executors
a=sc.accumulator(0)   #Creating an accumulator for adding values across executors
RDD0=sc.parallelize([y for y in range(0,5)]) #RDD from input python collection
RDD2=sc.parallelize([z for z in range(10,15)])
RDD1=RDD0.cartesian(RDD2) 
cached=RDD2.cache() #Testing cached RDD
RDD22=RDD1.map(lambda x:x[0]+x[1]+b.value[0])
RDD3=RDD22.repartition(5) # To trigger a new stage.
RDD4=RDD2.map(lambda x: 3*x-b.value[0])
RDD5=RDD3.filter(lambda x:x%2==0)
RDD6=RDD4.filter(lambda x:x%2!=0)
RDD7=RDD5.cartesian(RDD6)
RDD8=RDD7.flatMap(lambda x: [x[i] for i in range(0,2)])
RDD9=RDD8.union(cached)
ans=RDD9.reduce(lambda x,y: x+y) # Doing a simple sum on the random data.
print(ans)
def f(x):
    global a
    time.sleep(0.7) #Making the job run a little longer
    a+=x
RDD9.foreach(f)
print(a.value)

### Example: Spark Pi

In [ ]:
%%spark

from random import random
from operator import add

partitions = 10
n = 100000 * partitions

def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

count = sc.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))